In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import pickle
import requests
import re
import numpy as np

alphabet = 'abcdefghijklmnopqrstuvwxyz'.upper()
games = ['WC1','WC2','WC2X','WC3','WC3X','WoW','WoWBC','WoWWotLK','WoWC','WoWMoP','WoWWoD','WoWL','WoWBfA','WoWS','WoWD']
games2 = [1,
          2,
          3,
          4,
          5,
          'World of Warcraft',
          'The Burning Crusade',
          'Wrath of the Lich King',
          'Cataclysm',
          'Mists of Pandaria',
          'Warlords of Draenor',
          'Legion',
          'Battle for Azeroth', 
          'Shadowlands',
          'Dragonflight']


games_dict = {}
for i in range(len(games[5:])):
    games_dict[games2[i+5]] = games[i+5]
games_dict

{'World of Warcraft': 'WoW',
 'The Burning Crusade': 'WoWBC',
 'Wrath of the Lich King': 'WoWWotLK',
 'Cataclysm': 'WoWC',
 'Mists of Pandaria': 'WoWMoP',
 'Warlords of Draenor': 'WoWWoD',
 'Legion': 'WoWL',
 'Battle for Azeroth': 'WoWBfA',
 'Shadowlands': 'WoWS',
 'Dragonflight': 'WoWD'}

In [2]:
def get_importance(soup):
    colors = []

    i = 0
    soup_a = soup.select("ul > li")
    for a in soup_a:
        color = re.findall(r'color+:\w+', str(a))

        if len(color) > 0:
            if i > 3:
                colors.append(color[0][6:])
            i += 1
            
    return colors

In [3]:
def get_metadata():
    meta_data = []

    # get meta data of all major characters

    url = "https://wowpedia.fandom.com/wiki/Major_characters"
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    soup_a = soup.select("p > small")
    soup_b = soup.select("p > a")
    soup_b = [b for b in soup_b if "Katrana" not in str(b) and "Daval" not in str(b)]
    
    colors = get_importance(soup)
            
    for i,s in enumerate(soup_a):
        d = {}

        expansions = s.text.replace(",","").replace("—","").split()
        expansions = [int(1) if i in expansions else int(0) for i in games]

        text = soup_b[i]['href'][6:]
        color = colors[i]
        
        name = text.replace("%27", "'")

        if 1 in expansions:
            d['Name'] = name
            d['Importance'] = color
            for j,g in enumerate(games):
                d[g] = int(expansions[j])


            meta_data.append(d)
    
    return meta_data

In [4]:
url = "https://wowpedia.fandom.com/wiki/Archimonde"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")


In [52]:
# url = "https://wowpedia.fandom.com/wiki/Deathwing#Biography"
url = "https://wowpedia.fandom.com/wiki/Illidan_Stormrage#Biography"
# url = "https://wowpedia.fandom.com/wiki/Lord_Jaraxxus"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

text_dict = {}

for g in games_dict.keys():
    try:
        h = g.replace("The ", "") # get rid of "The" in "The Burning Crusade"
        soup_d = soup.find("h3", text=h)
        
        t = []
        include = True

        for sib in soup_d.find_next_siblings():
            if sib.name == "h3":
                break
            else:
                section = "This section concerns content related to "
                if section in sib.text.strip():
                    text = sib.text.strip()
                    game = text.replace(section, "").replace("the original ", "").replace(".", "")
                    if game not in games_dict.keys():
                        include = False
                        break
                    else:
                        print(game)
                        pass
                else:
                    if len(sib.text.strip()) > 0:
                        if sib.name == 'h2' and include == False:
                            pass
                        else:
                            t.append(sib.text.strip())

        text_dict[g] = t
    except AttributeError:
        pass


text_dict
    

Wrath of the Lich King
Cataclysm
Legion


{'Wrath of the Lich King': ['The duel with Arthas in Wrath of the Lich King.',
  'A vision of Illidan appeared as part of the quest \xa0[25-30]\xa0The Hunter and the Prince during a recreation of the duel between Illidan and Arthas Menethil at the gates of Icecrown Citadel.',
  'A "memory" of Illidan could be summoned by Argent Confessor Paletress in the Trial of the Champion.'],
 'Cataclysm': ["In Felwood, adventurers were able to relive Illidan's talk with Arthas, as well as the moment he obtained the Skull of Gul'dan, and finally his battle with Tichondrius, where they would help him against the dreadlord.[52]",
  "While visiting the past through the Caverns of Time, he made an appearance in the Well of Eternity dungeon and aided heroes in the battle against Mannoroth and Varo'then."],
 'Legion': ["Illidan discovered by Gul'dan.",
  "Illidan's soul in Helheim.",
  'Illidan after his resurrection.',
  "After the fall of the Black Temple, the corpse of Illidan was taken by Warden Maie

In [86]:
url = "https://wowpedia.fandom.com/wiki/Zovaal"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")


text_dict = {}

# h = g.replace("The ", "") # get rid of "The" in "The Burning Crusade"
soup_d = soup.find_all("span", {"class": "greybar"})

for d in soup_d:
    print(d.text)

This section concerns content related to the Warcraft novels, novellas, or short stories.
This section concerns content related to Shadowlands.
This section concerns content related to the Warcraft novels, novellas, or short stories.
This section concerns content related to Shadowlands.


In [72]:
soup_d = soup.find("h3", text="Wrath of the Lich King")

soup_d.text

'Wrath of the Lich King'

In [71]:
meta_data = get_metadata()

In [72]:
meta_data

[{'Name': 'Terokk',
  'Importance': 'yellow',
  'WC1': 0,
  'WC2': 0,
  'WC2X': 0,
  'WC3': 0,
  'WC3X': 0,
  'WoW': 0,
  'WoWBC': 1,
  'WoWWotLK': 0,
  'WoWC': 0,
  'WoWMoP': 0,
  'WoWWoD': 1,
  'WoWL': 0,
  'WoWBfA': 0,
  'WoWS': 0,
  'WoWD': 0},
 {'Name': 'Talon_King_Ikiss',
  'Importance': 'green',
  'WC1': 0,
  'WC2': 0,
  'WC2X': 0,
  'WC3': 0,
  'WC3X': 0,
  'WoW': 0,
  'WoWBC': 1,
  'WoWWotLK': 0,
  'WoWC': 0,
  'WoWMoP': 0,
  'WoWWoD': 1,
  'WoWL': 0,
  'WoWBfA': 0,
  'WoWS': 0,
  'WoWD': 0},
 {'Name': 'Archimonde',
  'Importance': 'red',
  'WC1': 0,
  'WC2': 0,
  'WC2X': 0,
  'WC3': 1,
  'WC3X': 0,
  'WoW': 0,
  'WoWBC': 1,
  'WoWWotLK': 1,
  'WoWC': 0,
  'WoWMoP': 0,
  'WoWWoD': 1,
  'WoWL': 1,
  'WoWBfA': 0,
  'WoWS': 0,
  'WoWD': 0},
 {'Name': "Kil'jaeden",
  'Importance': 'red',
  'WC1': 0,
  'WC2': 0,
  'WC2X': 0,
  'WC3': 0,
  'WC3X': 1,
  'WoW': 0,
  'WoWBC': 1,
  'WoWWotLK': 0,
  'WoWC': 0,
  'WoWMoP': 0,
  'WoWWoD': 1,
  'WoWL': 1,
  'WoWBfA': 0,
  'WoWS': 0,
  'WoWD

In [12]:
meta_data = get_metadata()
df_meta = pd.DataFrame(meta_data, dtype=object)
# saving the df's
with open('df_meta.pickle', 'wb') as handle:
    pickle.dump(df_meta, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_meta

,Name,WC1,WC2,WC2X,WC3,WC3X,WoW,WoWBC,WoWWotLK,WoWC,WoWMoP,WoWWoD,WoWL,WoWBfA,WoWS,WoWD
0,Terokk,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,Talon_King_Ikiss,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,Archimonde,0,0,0,1,0,0,1,1,0,0,1,1,0,0,0
3,Kil'jaeden,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0
4,Mannoroth,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,Vilnak'dor,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
268,Helya,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
269,Agatha,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
270,Annhylde_the_Caller,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [15]:
def get_characters():
    characters = []
    
    # get meta data of all major characters

    url = "https://wowpedia.fandom.com/wiki/Major_characters"
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    soup_a = soup.select("p > small")
    soup_b = soup.select("p > a")
    for i,s in enumerate(soup_a):
        d = {}

        expansions = s.text.replace(",","").replace("—","").split()
        expansions = [int(1) if i in expansions else int(0) for i in games]

        text = soup_b[i]['href'][6:]
        name = text.replace("%27", "'")

        if 1 in expansions:
            d['Name'] = name
            
            characters.append(d)
#     # get lore characters

#     c_list = []

#     for letter in tqdm(alphabet):
#         url = "https://wowpedia.fandom.com/wiki/Category:Lore_characters?pagefrom={}".format(letter)
#         html = urlopen(url).read()
#         soup = BeautifulSoup(html, features="html.parser")

#         for i in soup.find_all('li'):
#             text = i.find('a')['href']
#             verboten = ['/wiki/Genealogy', '/wiki/Local_Sitemap']
#             if text[0:5] == '/wiki' and ":" not in text and text not in verboten:
#                 c_list.append(text.replace("%27", "'").replace("%22", '"').split("/wiki/")[1])

#     c_list = sorted(list(set(c_list)))

#     for name in c_list:
#         d = {}
#         d['Name'] = name

#         characters.append(d)
    
    return characters

In [16]:
characters = get_characters()
df = pd.DataFrame(characters)
# saving the df's
with open('df.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)
df


,Name
0,Terokk
1,Talon_King_Ikiss
2,Archimonde
3,Kil'jaeden
4,Mannoroth
...,...
267,Zul
268,Talanji
269,Vilnak'dor
270,Helya


In [17]:
def get_wikicode(t):

    response = requests.get(
         'https://wowpedia.fandom.com/api.php',
             params={
             'action': 'query',
             'format': 'json',
             'titles': '{}'.format(t),
             'prop': 'revisions',
             'rvprop': 'content',
         }
     ).json()


    page = next(iter(response['query']['pages'].values()))

    wikicode = page['revisions'][0]['*']
    
    return wikicode

In [33]:
import pickle

with open('chars2.pickle', 'rb') as handle:
    characters = pickle.load(handle)
print(len(characters))

2333


In [19]:
# characters = []
with open('chars2.pickle', 'rb') as handle:
    characters = pickle.load(handle)

# for i,name in enumerate(tqdm(list(df['Name']))):
for name in tqdm(list(df['Name'].iloc[255:])):
    
#     if i % 20 == 0:
#         characters = list(np.unique(characters))
        
#         with open('characters.pickle', 'wb') as handle:
#             pickle.dump(characters, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
    text = get_wikicode(name)

    WORD_RE = re.compile(r"\[{2}.*?\]{2}")
    for word in list(np.unique(WORD_RE.findall(text))):
        try:
            char = word.replace(" ", "_")[2:-2]
            if "Category" not in char and ":" not in char and "#" not in char and "|" not in char:
                url = "https://wowpedia.fandom.com/wiki/{}".format(char)
                html = urlopen(url).read()
                soup = BeautifulSoup(html, features="html.parser")

                table =  soup.find("table",{"class":"infobox darktable"})

                if len(table) > 0:

                    text = table.text
                    if "Gender" not in text:
                        pass
                    else:
                        characters.append(char)

        except:
            pass

characters = list(np.unique(characters))

# df = df.append(pd.DataFrame(characters, columns=['Name']), ignore_index=True)

In [28]:
filter_characters = []

for char in characters:
    if "#" not in char and "|" not in char and "_(tactics)" not in char:
        filter_characters.append(char)

In [35]:
len(filter_characters)

2253

In [36]:
df = df.append(pd.DataFrame(filter_characters, columns=['Name']), ignore_index=True)

In [32]:
list(asdasdas['Name'])

['Terokk',
 'Talon_King_Ikiss',
 'Archimonde',
 "Kil'jaeden",
 'Mannoroth',
 'Balnazzar',
 'Magtheridon',
 "Mal'Ganis",
 'Tichondrius',
 'Varimathras',
 'Azgalor',
 'Hakkar_the_Houndmaster',
 'Kazzak',
 'Velen',
 'Akama',
 'Nobundo',
 'Vindicator_Maraad',
 'Yrel',
 'Alexstrasza',
 'Kalecgos',
 'Malygos',
 'Deathwing',
 'Daval_Prestor',
 'Nozdormu',
 'Ysera',
 'Korialstrasz',
 'Eranikus',
 'Sintharia',
 'Tyranastrasz',
 'Nefarian',
 'Onyxia',
 'Katrana_Prestor',
 'Wrathion',
 'Sabellian',
 'Arygos',
 'Kirygosa',
 'Chronormu',
 'Galakrond',
 'Kairozdormu',
 'Tyrygosa',
 'Sapphiron',
 'Ebyssian',
 'Dargrul',
 'Navarrogg',
 'Brann_Bronzebeard',
 'Muradin_Bronzebeard',
 'Magni_Bronzebeard',
 'Moira_Thaurissan',
 'Dagran_Thaurissan_II',
 'Thargas_Anvilmar',
 'Falstad_Wildhammer',
 'Kurdran_Wildhammer',
 'Dagran_Thaurissan',
 'Sorcerer-thane_Thaurissan',
 "Kael'thas_Sunstrider",
 'Aethas_Sunreaver',
 'Grand_Magister_Rommath',
 'Halduron_Brightwing',
 'Lady_Liadrin',
 "Lor'themar_Theron",
 'Va

In [ ]:
# saving the df's
with open('df.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)